# Мониторинг агента на основе Middleware
- Decorator-based - быстрый и простой метод
- Class-based - более сложная и мощная система

In [1]:
from langchain.agents.middleware import before_model, after_model, wrap_model_call
from langchain.agents.middleware import AgentState, ModelRequest, ModelResponse, dynamic_prompt
from langchain.messages import AIMessage
from langchain.agents import create_agent
from langgraph.runtime import Runtime
from typing import Any, Callable
from langchain.tools.tool_node import ToolCallRequest
from langchain.agents.middleware import AgentMiddleware
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain.messages import HumanMessage, AIMessage, ToolMessage

In [2]:
def pretty_print(state):
    print("Printing messages")
    for message in state["messages"]:
        message.pretty_print()

In [3]:
# Логируем события перед вызовом модели
@before_model
def log_before_model(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print(f"About to call model with {len(state['messages'])} messages")
    return None


In [4]:
# Логируем события после вызова модели
@after_model(can_jump_to=["end"])
def validate_output(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print(f"Response contains of {len(state["messages"])} messages")
    return None

In [5]:
# Мониторинг вызова инструментов. Пример на основе класса 
class ToolMonitoringMiddleware(AgentMiddleware):
    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command],
    ) -> ToolMessage | Command:
        print(f"Executing tool: {request.tool_call['name']}")
        print(f"Arguments: {request.tool_call['args']}")

        try:
            result = handler(request)
            print(f"Tool completed successfully")
            return result
        except Exception as e:
            print(f"Tool failed: {e}")
            raise

In [6]:
# Tools
@tool
def get_weather(city: str):
    """Get weather information for a city."""
    return f"It's 75 degrees and sunny in {city}."

In [7]:
model = ChatOllama(model="gpt-oss:20b-cloud")

agent = create_agent(
    model=model,
    middleware=[log_before_model, validate_output, ToolMonitoringMiddleware()],
    tools=[get_weather],
)

In [8]:
result = agent.invoke({
        "messages": [HumanMessage(content="What's the weather in San Francisco?")]
    })

About to call model with 1 messages
Response contains of 2 messages
Executing tool: get_weather
Arguments: {'city': 'San Francisco'}
Tool completed successfully
About to call model with 3 messages
Response contains of 4 messages


In [11]:
pretty_print(result)

Printing messages
================================ Human Message =================================

What's the weather in San Francisco?
================================== Ai Message ==================================
Tool Calls:
  get_weather (d68c0a64-9c64-40be-812c-556444ea2aac)
 Call ID: d68c0a64-9c64-40be-812c-556444ea2aac
  Args:
    city: San Francisco
================================= Tool Message =================================
Name: get_weather

It's 75 degrees and sunny in San Francisco.
================================== Ai Message ==================================

The weather in San Francisco right now is **75 °F and sunny**.
